In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import os
import shutil
from pathlib import Path
import cv2
from glob import glob
from PIL import Image
from keras import layers
from keras import models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold

# 데이콘 이상치 탐색

**https://dacon.io/competitions/official/235894/overview/description**

In [4]:
path = "C:/Users/choij/Desktop/datasets/open/open/train/train/"
#train dataset 위치

In [5]:
path2 = 'C:/Users/choij/Desktop/datasets/open/open/test/test/'
#test 데이터셋 위치

## 데이터 전처리

In [8]:
df = pd.read_csv("C:/Users/choij/Desktop/datasets/open/open/train_df.csv")
df #1024 * 1024 * 3 형식의 이미지들

,index,file_name,class,state,label
0,0,10000.png,transistor,good,transistor-good
1,1,10001.png,capsule,good,capsule-good
2,2,10002.png,transistor,good,transistor-good
3,3,10003.png,wood,good,wood-good
4,4,10004.png,bottle,good,bottle-good
...,...,...,...,...,...
4272,4272,14272.png,transistor,good,transistor-good
4273,4273,14273.png,transistor,good,transistor-good
4274,4274,14274.png,grid,good,grid-good
4275,4275,14275.png,zipper,good,zipper-good


In [9]:
print(len(df['state'].value_counts())) 
print(len(df['class'].value_counts())) 
print(len(df['label'].value_counts())) # 88개의 클래스 분류

49
15
88


In [10]:
df['label'].value_counts()

hazelnut-good              391
screw-good                 320
carpet-good                280
pill-good                  267
grid-good                  264
                          ... 
cable-poke_insulation        5
pill-pill_type               5
transistor-damaged_case      5
transistor-bent_lead         5
wood-color                   4
Name: label, Length: 88, dtype: int64

In [11]:
df = df.drop(['class','state' , 'file_name'], axis = 1)

In [12]:
df.head()

,index,label
0,0,transistor-good
1,1,capsule-good
2,2,transistor-good
3,3,wood-good
4,4,bottle-good


In [13]:
categories = np.unique(df['label'])
categories #클래스 집합 리스트

array(['bottle-broken_large', 'bottle-broken_small',
       'bottle-contamination', 'bottle-good', 'cable-bent_wire',
       'cable-cable_swap', 'cable-combined', 'cable-cut_inner_insulation',
       'cable-cut_outer_insulation', 'cable-good', 'cable-missing_cable',
       'cable-missing_wire', 'cable-poke_insulation', 'capsule-crack',
       'capsule-faulty_imprint', 'capsule-good', 'capsule-poke',
       'capsule-scratch', 'capsule-squeeze', 'carpet-color', 'carpet-cut',
       'carpet-good', 'carpet-hole', 'carpet-metal_contamination',
       'carpet-thread', 'grid-bent', 'grid-broken', 'grid-glue',
       'grid-good', 'grid-metal_contamination', 'grid-thread',
       'hazelnut-crack', 'hazelnut-cut', 'hazelnut-good', 'hazelnut-hole',
       'hazelnut-print', 'leather-color', 'leather-cut', 'leather-fold',
       'leather-glue', 'leather-good', 'leather-poke', 'metal_nut-bent',
       'metal_nut-color', 'metal_nut-flip', 'metal_nut-good',
       'metal_nut-scratch', 'pill-color', 'p

In [14]:
nb_classes = len(categories)
nb_classes
#https://velog.io/@jaehyeong/CNN-%EB%AA%A8%EB%8D%B8%EC%9D%84-%ED%86%B5%ED%95%9C-%EC%9E%90%EB%8F%99%EC%B0%A8-%EC%82%AC%EA%B3%A0-%EC%9D%B4%EB%AF%B8%EC%A7%80-%EB%B6%84%EB%A5%98

88

In [15]:
base_dir = "C:/Users/choij/Desktop/datasets/open/open/train/small_train"

In [16]:
test_base_dir = "C:/Users/choij/Desktop/datasets/open/open/test/small_test"

In [79]:
os.mkdir(base_dir)

In [121]:
os.mkdir(test_base_dir)

In [17]:
fname_list = os.listdir(base_dir)

In [18]:
base = base_dir + '/'
base

'C:/Users/choij/Desktop/datasets/open/open/train/small_train/'

In [19]:
test_base = test_base_dir + '/'

In [80]:
for fname in categories:
    fnames = os.path.join(base_dir, fname)
    os.mkdir(fnames) # class 별로 폴더 생성

In [124]:
for fname in categories:
    fnames = os.path.join(test_base_dir, fname)
    os.mkdir(fnames)

In [20]:
format = [".jpg",".png",".jpeg","bmp",".JPG",".PNG","JPEG","BMP"]

In [50]:
for (path, dirs, files) in os.walk(path):
    for file in files:
        if file.endswith(tuple(format)):
            image = Image.open(path+"/"+file)
            image=image.resize((int(image.size[0]*0.2), int(image.size[1]*0.2)))
            image.save(path+"/"+file)
        else:
            print(path)
            print("InValid",file) # 훈련 파일크기 작게

In [51]:
for (path, dirs, files) in os.walk(path2):
    for file in files:
        if file.endswith(tuple(format)):
            image = Image.open(path+"/"+file)
            image=image.resize((int(image.size[0]*0.2), int(image.size[1]*0.2)))
            image.save(path+"/"+file)

        else:
            print(path)
            print("InValid",file) # 테스트 파일크기 작게

In [84]:
for idx in range(len(os.listdir(path))): #0~4277 반복하면서 훈련 이미지 복사
    index = 10000
    for fname in fname_list: #폴더 리스트 
        if df['label'][idx] == fname:
            index += idx
            index = str(index) + '.png' 
            path_dir = base_dir + '/'+ fname
            shutil.copyfile(path+index, path_dir + '/' + str(index) + '.png')

In [132]:
for idx in range(len(os.listdir(path2))): #0~4277 반복 테스트 이미지 복사
    index = 20000
    for fname in fname_list: #폴더 리스트 1개씩 흘음 
        if df['label'][idx] == fname:
            index += idx
            index = str(index) + '.png' 
            path_dir = test_base_dir + '/'+ fname
            shutil.copyfile(path2+index, path_dir + '/' + str(index))

In [22]:
def CustomModel():
    model = models.Sequential()

    model.add(layers.Conv2D(32, (5,5), activation = 'relu', input_shape = (150,150,3)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))

    model.add(layers.Conv2D(32, (5,5), activation = 'relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))

    model.add(layers.Conv2D(64, (5,5), activation = 'relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))

    model.add(layers.Conv2D(64, (1,1), activation = 'relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))

    model.add(layers.Conv2D(128, (5,5), activation = 'relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation = 'relu'))
    model.add(layers.Dense(88, activation = 'softmax'))
    return model

In [52]:
image_w = 150
image_h = 150

X = []
Y = []

for idx, cat in enumerate(categories):
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    
    files = glob(base_dir + '/' + cat + '/*.png')
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert('RGB')
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        X.append(data)
        Y.append(label)

In [24]:
X = np.array(X)
Y = np.array(Y)

print(X.shape)
print(Y.shape)

(4277, 150, 150, 3)
(4277, 88)


In [25]:
x_train = X
y_train = Y

In [53]:
image_w = 150
image_h = 150

test_X = []
test_Y = []

for idx, cat in enumerate(categories):
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    
    files = glob(test_base_dir + '/' + cat + '/*.png')
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert('RGB')
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        test_X.append(data)
        test_Y.append(label)

In [27]:
test_x = np.array(test_X)
y_test = np.array(test_Y)

print(test_x.shape)
print(y_test.shape)

(2154, 150, 150, 3)
(2154, 88)


In [28]:
x_train = x_train.astype('float') / 255
x_test = test_x.astype('float') / 255

In [29]:
y_train.argmax(1)

array([ 0,  0,  0, ..., 87, 87, 87], dtype=int64)

In [30]:
print(x_train.shape)
print(y_train.shape)

(4277, 150, 150, 3)
(4277, 88)


In [54]:
for i,fname in enumerate(os.listdir(base_dir)):
    print('클래스:{}, 이미지 수 :{}'.format(i, len(os.listdir(base_dir + '/'+fname))))

클래스:0, 이미지 수 :10
클래스:1, 이미지 수 :11
클래스:2, 이미지 수 :11
클래스:3, 이미지 수 :209
클래스:4, 이미지 수 :7
클래스:5, 이미지 수 :6
클래스:6, 이미지 수 :6
클래스:7, 이미지 수 :7
클래스:8, 이미지 수 :5
클래스:9, 이미지 수 :224
클래스:10, 이미지 수 :6
클래스:11, 이미지 수 :5
클래스:12, 이미지 수 :5
클래스:13, 이미지 수 :12
클래스:14, 이미지 수 :11
클래스:15, 이미지 수 :219
클래스:16, 이미지 수 :11
클래스:17, 이미지 수 :12
클래스:18, 이미지 수 :10
클래스:19, 이미지 수 :10
클래스:20, 이미지 수 :9
클래스:21, 이미지 수 :280
클래스:22, 이미지 수 :9
클래스:23, 이미지 수 :9
클래스:24, 이미지 수 :10
클래스:25, 이미지 수 :6
클래스:26, 이미지 수 :6
클래스:27, 이미지 수 :6
클래스:28, 이미지 수 :264
클래스:29, 이미지 수 :6
클래스:30, 이미지 수 :6
클래스:31, 이미지 수 :9
클래스:32, 이미지 수 :9
클래스:33, 이미지 수 :391
클래스:34, 이미지 수 :9
클래스:35, 이미지 수 :9
클래스:36, 이미지 수 :10
클래스:37, 이미지 수 :10
클래스:38, 이미지 수 :9
클래스:39, 이미지 수 :10
클래스:40, 이미지 수 :245
클래스:41, 이미지 수 :9
클래스:42, 이미지 수 :13
클래스:43, 이미지 수 :11
클래스:44, 이미지 수 :12
클래스:45, 이미지 수 :220
클래스:46, 이미지 수 :12
클래스:47, 이미지 수 :13
클래스:48, 이미지 수 :9
클래스:49, 이미지 수 :11
클래스:50, 이미지 수 :13
클래스:51, 이미지 수 :10
클래스:52, 이미지 수 :267
클래스:53, 이미지 수 :5
클래스:54, 이미지 수 :12
클래스:55, 이미지 수 :320
클래스:56, 이미지 수 :1

In [46]:
for i,fname in enumerate(os.listdir(test_base_dir)):
    print('클래스:{}, 이미지 수 :{}'.format(i, len(os.listdir(test_base_dir + '/'+fname))))
        

클래스:0, 이미지 수 :8
클래스:1, 이미지 수 :7
클래스:2, 이미지 수 :6
클래스:3, 이미지 수 :112
클래스:4, 이미지 수 :4
클래스:5, 이미지 수 :3
클래스:6, 이미지 수 :2
클래스:7, 이미지 수 :4
클래스:8, 이미지 수 :3
클래스:9, 이미지 수 :103
클래스:10, 이미지 수 :4
클래스:11, 이미지 수 :2
클래스:12, 이미지 수 :3
클래스:13, 이미지 수 :6
클래스:14, 이미지 수 :6
클래스:15, 이미지 수 :110
클래스:16, 이미지 수 :5
클래스:17, 이미지 수 :5
클래스:18, 이미지 수 :4
클래스:19, 이미지 수 :6
클래스:20, 이미지 수 :4
클래스:21, 이미지 수 :147
클래스:22, 이미지 수 :5
클래스:23, 이미지 수 :7
클래스:24, 이미지 수 :4
클래스:25, 이미지 수 :2
클래스:26, 이미지 수 :3
클래스:27, 이미지 수 :4
클래스:28, 이미지 수 :131
클래스:29, 이미지 수 :2
클래스:30, 이미지 수 :4
클래스:31, 이미지 수 :3
클래스:32, 이미지 수 :5
클래스:33, 이미지 수 :185
클래스:34, 이미지 수 :6
클래스:35, 이미지 수 :4
클래스:36, 이미지 수 :4
클래스:37, 이미지 수 :6
클래스:38, 이미지 수 :2
클래스:39, 이미지 수 :5
클래스:40, 이미지 수 :125
클래스:41, 이미지 수 :4
클래스:42, 이미지 수 :7
클래스:43, 이미지 수 :6
클래스:44, 이미지 수 :5
클래스:45, 이미지 수 :127
클래스:46, 이미지 수 :4
클래스:47, 이미지 수 :8
클래스:48, 이미지 수 :4
클래스:49, 이미지 수 :6
클래스:50, 이미지 수 :6
클래스:51, 이미지 수 :8
클래스:52, 이미지 수 :130
클래스:53, 이미지 수 :4
클래스:54, 이미지 수 :7
클래스:55, 이미지 수 :153
클래스:56, 이미지 수 :8
클래스:57, 이미지 수 :5
클래스:

In [48]:
skf = StratifiedKFold(n_splits = 15, random_state = 32, shuffle = True)
n = 0

cnn_pred = []

for train_index, valid_index in skf.split(x_train, y_train.argmax(1)):
    n += 1
    x_train, x_valid = x_train[train_index], x_train[valid_index]
    y_train, y_valid = y_train[train_index], y_train[valid_index]
    
    model = CustomModel()
    
    model.compile(optimizer = 'Adam', 
                  loss = 'categorical_crossentropy', 
                  metrics = ['accuracy'])
    
    filename = 'CNN_checkpoint3.h5'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filename,
                                                   monitor = 'val_accuracy',
                                                   verbose = 0,
                                                   save_best_only = True,
                                                   mode = 'auto')

    earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                    patience = 6)
    
    train_datagen = ImageDataGenerator(rescale = 1./255,
                                        rotation_range = 40,
                                        width_shift_range = 0.2,
                                        height_shift_range = 0.2,
                                        shear_range = 0.2,
                                        zoom_range = 0.2,
                                        horizontal_flip = True)
    
    test_datagen = ImageDataGenerator(rescale = 1./255)
    
    valid_datagen = ImageDataGenerator(rescale = 1./255)

    train_generator = train_datagen.flow_from_directory(
        base_dir,
        target_size = (150,150),
        batch_size = 40,
        class_mode = 'categorical')
    
    valid_generator = test_datagen.flow_from_directory(
        test_base_dir,
        target_size = (150,150),
        batch_size = 40,
        class_mode = 'categorical')
        
    history = model.fit_generator(
    train_generator,
    steps_per_epoch = 100,
    epochs = 10,
    validation_data = valid_generator,
    validation_steps = 40,
    callbacks = [checkpoint, earlystopping])
    

C:\Users\choij\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=15.
  warnings.warn(("The least populated class in y has only %d"


Found 4277 images belonging to 88 classes.
Found 2154 images belonging to 88 classes.


<ipython-input-48-5a6faa8afd4c>:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
100/100 [==============================] - 158s 2s/step - loss: 1.7534 - accuracy: 0.7043 - val_loss: 4.8242 - val_accuracy: 0.0675
Epoch 2/10
100/100 [==============================] - 146s 1s/step - loss: 1.0871 - accuracy: 0.8224 - val_loss: 5.3524 - val_accuracy: 0.0669
Epoch 3/10
100/100 [==============================] - 149s 1s/step - loss: 0.9096 - accuracy: 0.8359 - val_loss: 6.2336 - val_accuracy: 0.0694
Epoch 4/10
100/100 [==============================] - 154s 2s/step - loss: 0.8550 - accuracy: 0.8351 - val_loss: 6.6606 - val_accuracy: 0.0594
Epoch 5/10
100/100 [==============================] - 147s 1s/step - loss: 0.7997 - accuracy: 0.8401 - val_loss: 7.4565 - val_accuracy: 0.0512
Epoch 6/10
100/100 [==============================] - 147s 1s/step - loss: 0.7912 - accuracy: 0.8394 - val_loss: 10.4868 - val_accuracy: 0.0581
Epoch 7/10
100/100 [==============================] - 149s 1s/step - loss: 0.7595 - accuracy: 0.8414 - val_loss: 9.2524 - val_accuracy: 0.052

IndexError: index 3724 is out of bounds for axis 0 with size 3724